# 2 Simulating Price and Funding Fees

1. Compared to the volatility of the underlying, what is the likelihood that I will profit in this scenario? (Portfolio Size > 1)
   1. How volatile should the underlying be to make a profit? Is there an Analytical Solution?
   2. What is the likelihood that a leverage long position will make money?
   3. How different would the funding payoff be?
2. What is the expected amount that I will pay for cryptocurrencies? - screen through all possible tokens
3. What is the difference between me opening a position on Hyperliquid vs Binance?
   1. Notional
   2. Percentage per hour 
4. What is the optimal platform?
<!-- 7. How can I scale it to other platforms? -->

## Assumptions
- 2 Independent Random Distributions (Returns and Funding Fees)
- Spot price mirrors Perpetual - premium does not matter

## Future Extension
- Premium difference
- Max leverage varies by asset, ranging from 3x to 50x. Maintenance margin is half of the initial margin at max leverage. E.g., if max leverage is 20x, the maintenance margin is 2.5%.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import json

from datetime import datetime, timedelta
import pytz

import scipy.stats as scipy_stats

# Binance Prices

In [5]:
def get_binance_spot(symbol, interval, startTime, endTime):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": startTime,
        "endTime": endTime,
        "timeZone": 0,
        "limit": 1000
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

startTime = int(datetime(2025, 1,1 , tzinfo=pytz.UTC).timestamp() * 1000)
endTime = int(datetime.now(pytz.UTC).timestamp() * 1000)

data = get_binance_spot("BTCUSDT", "1d", startTime=startTime, endTime=endTime)
data

[[1735689600000,
  '93576.00000000',
  '95151.15000000',
  '92888.00000000',
  '94591.79000000',
  '10373.32613000',
  1735775999999,
  '975444194.13799830',
  1516556,
  '5347.73648000',
  '502914035.64059070',
  '0'],
 [1735776000000,
  '94591.78000000',
  '97839.50000000',
  '94392.00000000',
  '96984.79000000',
  '21970.48948000',
  1735862399999,
  '2118411852.68127950',
  3569079,
  '10915.96986000',
  '1052226710.33367170',
  '0'],
 [1735862400000,
  '96984.79000000',
  '98976.91000000',
  '96100.01000000',
  '98174.18000000',
  '15253.82936000',
  1735948799999,
  '1486610768.59079170',
  2851164,
  '7468.60797000',
  '727787246.89655880',
  '0'],
 [1735948800000,
  '98174.17000000',
  '98778.43000000',
  '97514.79000000',
  '98220.50000000',
  '8990.05651000',
  1736035199999,
  '881223751.63536030',
  1559982,
  '4350.80221000',
  '426524714.13055390',
  '0'],
 [1736035200000,
  '98220.51000000',
  '98836.85000000',
  '97276.79000000',
  '98363.61000000',
  '8095.63723000',
 

# Bybit SPOT

In [3]:
def get_bybit_kline(category, symbol, interval, start, end): 
    url = " https://api.bybit.com/v5/market/kline"
    params = {
        "category": category,
        "symbol": symbol,
        "interval": interval,
        "from": start,
        "to": end,
        "limit": 200
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

start = int(datetime(2025, 1,1 , tzinfo=pytz.UTC).timestamp() * 1000)
end = int(datetime.now(pytz.UTC).timestamp() * 1000)

data = get_bybit_kline(category="spot", symbol= "BTCUSDT", interval="60", start=start, end=end)
data

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'spot',
  'symbol': 'BTCUSDT',
  'list': [['1738548000000',
    '93433.04',
    '94954.87',
    '91201',
    '93543',
    '8650.968721',
    '807796123.73996179'],
   ['1738544400000',
    '96491.82',
    '96921.02',
    '91888',
    '93433.04',
    '10433.183992',
    '981442579.95207113'],
   ['1738540800000',
    '97690.9',
    '97774.22',
    '95441.24',
    '96491.82',
    '5652.76663',
    '546316882.48214375'],
   ['1738537200000',
    '98009.68',
    '98015.85',
    '96537.48',
    '97690.9',
    '2635.932361',
    '256580636.85890139'],
   ['1738533600000',
    '97036.69',
    '98030.6',
    '96126.87',
    '98009.68',
    '1508.505497',
    '146496674.07472688'],
   ['1738530000000',
    '97786.27',
    '97979.15',
    '96561.01',
    '97036.69',
    '1412.083712',
    '137215264.78787696'],
   ['1738526400000',
    '98260.09',
    '98627.21',
    '97652.38',
    '97786.27',
    '904.637837',
    '88812786.88993305'],
  

In [4]:
requests.get(url = "https://api.bybit.com/v5/market/time")

<Response [200]>